In [1]:
# import dependencies
import pandas as pd
import math


In [2]:
# read csv on carbon emission data and convert to pandas dataframe
carbon = pd.read_csv('../resources/data/carbon_by_country.csv')

# select only emissions in gigatons of carbon (GtC) by USA since 1969
carbon = carbon[['year', 'USA']].loc[carbon['year'] >= 1969,:]

# rename 'USA' column to 'emissions(GtC)'
carbon = carbon.rename(columns={'USA': 'emissions(GtC)'})

# reset index
carbon = carbon.reset_index(drop=True)


In [3]:
# create funciton to calculate total accumulated carbon emissions(GtC)
def total(x):
    return carbon.loc[carbon['year'] < x,:]['emissions(GtC)'].sum()

# create column with total accumulated carbon emissions for each year
carbon['accumulated(GtC)'] = carbon['year'].apply(total)


In [4]:
# create funciton to calculate scaled dft size
def scale(x):
    return (x-0)/(70730-0)*(524288-16)+16

# create column with scaled dft size for each year
carbon['dft_size'] = carbon['accumulated(GtC)'].apply(scale)


In [5]:
# create funciton to calculate viable fft size
def fft_size(x):
    return 1 if x == 0 else int(2**math.floor(math.log2(x)))

# create column with viable fft size for each year
carbon['fft_size'] = carbon['dft_size'].apply(fft_size)


In [6]:
# create funciton to calculate frequency resolution
def freq_res(x):
    return 44100/x

# create column with frequency resoulution for each year
carbon['freq_res(Hz)'] = carbon['fft_size'].apply(freq_res)


In [7]:
# create funciton to calculate file names
def file(x):
    
    # all unique fft sizes used
    ffts = carbon['fft_size'].unique()
    
    # iterate through unique fft sizes
    for i in range(0, len(ffts)):
        
        # if the year's fft size matches the indexed fft size:
        if x == ffts[i]:
            
            # return the file name with the number as the index plus 1
            return f'jimi{i+1}.mp3'

# create column with file name for each year
carbon['file'] = carbon['fft_size'].apply(file)


In [8]:
# create function to calculate starting times in milliseconds(ms) for each file
def begin(x):
    
    # number of years represented by previous files
    previous = 0
    
    # iterate through previous files
    for i in range(0, int(x[4])):
        
        # add the number of years represented by each previous file to previous
        previous = previous + len(carbon.loc[carbon['file'] == f'jimi{i}.mp3',:])

    # return start time as previous years multiplied by scaled length of a year in audio playback(ms)
    return previous*4735

# create column for starting times of each file
carbon['begin(ms)'] = carbon['file'].apply(begin)


In [9]:
# create function to calculate ending times(ms) for each file
def end(x):
    
    # last file ends at ending of manipulated audio
    if x == 'jimi8.mp3':
        return 241471
    
    # for all other files:
    else:
        # number of years represented by current file
        years = len(carbon.loc[carbon["file"] == x,:])
        
        # final year represented by a file is 10 seconds to allow crossfading into next year
        final_year = 10000
        
        # other years multiplied by scaled length of a year in audio playback(ms)
        other_years = (years-1)*4735
        
        # return end time as begining time plus scaled length of years represented by file
        return begin(x) + final_year + other_years

# create column for ending times of each file
carbon['end(ms)'] = carbon['file'].apply(end)


In [10]:
# display full dataframe
carbon


,year,emissions(GtC),accumulated(GtC),dft_size,fft_size,freq_res(Hz),file,begin(ms),end(ms)
0,1969,1098.0,0.0,16.000000,16,2756.250000,jimi1.mp3,0,10000
1,1970,1181.0,1098.0,8154.705726,4096,10.766602,jimi2.mp3,4735,14735
2,1971,1188.0,2279.0,16908.632377,16384,2.691650,jimi3.mp3,9470,24205
3,1972,1244.0,3467.0,25714.445129,16384,2.691650,jimi3.mp3,9470,24205
4,1973,1300.0,4711.0,34935.346699,32768,1.345825,jimi4.mp3,18940,43145
5,1974,1254.0,6011.0,44571.337085,32768,1.345825,jimi4.mp3,18940,43145
6,1975,1201.0,7265.0,53866.361657,32768,1.345825,jimi4.mp3,18940,43145
7,1976,1257.0,8466.0,62768.534314,32768,1.345825,jimi4.mp3,18940,43145
8,1977,1292.0,9723.0,72085.795787,65536,0.672913,jimi5.mp3,37880,76290
9,1978,1333.0,11015.0,81662.487770,65536,0.672913,jimi5.mp3,37880,76290


In [32]:
# group years by file name
files_group = carbon.groupby('file')

# create dictionary to hold data for files
files = {}

# columns to extract data from
columns = ['fft_size', 'freq_res(Hz)', 'begin(ms)', 'end(ms)']

# iterate through columns
for c in columns:
    
    # create key to hold list with data for each column
    files[c] = []
    
    # append each column's data points to its respective list
    for u in files_group[c].unique():
        files[c].append(u[0])

# create dataframe out of files dictionary
files = pd.DataFrame(files)

# set files index as file name
files = files.set_index(carbon['file'].unique())

# display dataframe
files


,fft_size,freq_res(Hz),begin(ms),end(ms)
jimi1.mp3,16,2756.250000,0,10000
jimi2.mp3,4096,10.766602,4735,14735
jimi3.mp3,16384,2.691650,9470,24205
jimi4.mp3,32768,1.345825,18940,43145
jimi5.mp3,65536,0.672913,37880,76290
jimi6.mp3,131072,0.336456,71025,137845
jimi7.mp3,262144,0.168228,132580,242015
jimi8.mp3,524288,0.084114,236750,241471


In [15]:
# export dataframe as csv
# carbon.to_csv('../resources/data/carbon.csv')
